### Group 100 -  Conversational AI Assignment 2


```
1. Amit Kumar Sharma      2023ac05454       100%

2. Mohammed Faisal Sait   2023aa05525       100%

3. Chachiya Faiz Arif     2023ac05420       100%

4. Parveen Kumar          2023ac05467       100%

5. Sachchinda Nand Singh  2023ac05002       100%
```

## Objective
```Develop and compare two systems for answering questions based on company financial statements (last two years):

Retrieval-Augmented Generation (RAG) Chatbot: Combines document retrieval and generative response.
Fine-Tuned Language Model (FT) Chatbot: Directly fine-tunes a small open-source language model on financial Q&A.
Use the same financial data for both methods and perform a detailed comparison on accuracy, speed, and robustness.

### 1. Data Collection & Preprocessing
```Obtain financial statements for the last two years (publicly available or from a group member’s company).
Convert documents (PDF, Excel, HTML) to plain text using OCR or appropriate parsers.
Clean text by removing noise like headers, footers, and page numbers.
Segment reports into logical sections (e.g., income statement, balance sheet).
Construct at least 50 question-answer (Q/A) pairs reflecting the financial data.
Example:
Q: What was the company’s revenue in 2023?
A: The company’s revenue in 2023 was $4.13 billion.

In [ ]:
# %brew install poppler tesseract libmagic

import os
import re
import fitz 
from PIL import Image
from io import BytesIO

# %pip install -Uq "unstructured [all-docs]" pillow lxml spip install -Uq chromadb tiktoken
# %pip install -Uq langchain langchain-community langchain-openai langchain-groq
# %pip install -Uq python_dotenv
# %pip install pytesseract
# %pip install easyocr
import pytesseract
import re
import csv
import pdfplumber
import os
import pdfplumber
import pandas as pd
import fitz 
import easyocr

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-2.2.6-cp311-cp311-macosx_14_0_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-macosx_14_0_x86_64.whl (6.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.10 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.11.7 which is incompatible.
spacy 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/envs/convAI/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/envs/convAI/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/envs/convAI/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File 

### Added Financial Results of SAP SE for year 2023, 2024 
```under ../data folder
NYSE_SAP_2023.pdf,NYSE_SAP_2024.pdf

### Adding list of sections to be extracted

In [2]:
section_table_headers = [
        r'Key Facts About SAP Stock/SAP ADRs',
        r'Total Shareholder Return (Dividends Reinvested)',
        r'Meeting Attendance of SAP Supervisory Board Members During Fiscal Year',
        r'Reconciliation of Non-IFRS Revenue',
        r'Reconciliation of Non-IFRS Operating Expenses',
        r'Reconciliation of Non-IFRS Profit Numbers, Income Tax Expense, and Key Ratios',
        r'Reconciliation of Operating Cash Flow from Continuing Operations to Free Cash Flow',
        r'Non-IFRS Operating Expense Adjustments by Functional Areas',
        r'Comparison of Outlook and Results',
        r'Profit After Tax and Earnings per Share',
        r'Development of Net Debt',
        r'Analysis of Consolidated Statements of Cash Flow',
        r'SAP SE Income Statement − German Commercial Code',
        r'Assets and Financial Position',
        r'Risk Factors',
        r'Overview of Risk Factors',
        r'Economic, Political, Social, and Regulatory Risks',
        r'Economic Trends – GDP Growth Year Over Year',
        r'Capital Expenditures (CapEx)',
        r'Operational Expenditure (OpEx)',
        r'Integrated Report',
        r'Energy Consumption and Mix',
        r'Metrics on Material Topics',
        r'People Development',
        r'Responsible AI',
        r'Consolidated Statements of Financial Position',
        r'Consolidated Statements of Changes in Equity',
        r'Consolidated Statements of Cash Flows',
        r'Revenue',
        r'Operating Profit',
        r'Total Revenue by Region',
        r'Major Revenue Classes by Region',
        r'Reconciliation of Remaining Performance Obligations',
        r'Trade and Other Receivables',
        r'Employee Benefits Expenses',
        r'Employee Headcount by Region and Function',
        r'Equity-Settled Share-Based Payments',
        r'Changes in Outstanding Awards',
        r'Changes in Outstanding Awards',
        r'Restructuring Expenses by Functional Area',
        r'Reconciliation of Segment Measures to the Consolidated Income Statements',
        r'Other Non-Operating Income/Expense, Net',
        r'Financial Income, Net',
        r'Tax Expense by Geographic Location',
        r'Major Components of Tax Expense',
        r'Profit Before Tax by Geographic Location',
        r'Relationship Between Tax Expense and Profit Before Tax',
        r'Components of Recognized Deferred Tax Assets and Liabilities',
        r'Movement of Deferred Tax Assets and Liabilities',
        r'Items Not Resulting in a Deferred Tax Asset',
        r'Earnings per Share',
        r'Intangible Assets',
        r'Leases in the Balance Sheet',
        r'Cash and Cash Equivalents',
        r'Group Liquidity and Net Debt',
        r'Non-Derivative Financial Debt Investments',
        r'Financial Debt',
        r'Bonds',
        r'Private Placements',
        r'Reconciliation of Liabilities Arising from Financing Activities',
        r'Designated Hedged Items in Foreign Currency Exchange Rate Hedges',
        r'Designated Hedging Instruments in Foreign Currency Exchange Rate Hedges',
        r'Details on Hedging Instruments in Foreign Currency Exchange Rate Hedges',
        r'Foreign Currency Exchange Rate Exposure',
        r'Foreign Currency Sensitivity',
        r'Designated Hedged Items in Interest Rate Hedges',
        r'Designated Hedging Instruments in Interest Rate Hedges',
        r'Details on Hedging Instruments in Interest Rate Hedges',
        r'Interest Rate Exposure',
        r'Credit Risk Exposure',
        r'Master Netting and Similar Arrangements',
        r'Contractual Maturities of Non-Derivative Financial Liabilities',
        r'Fair Values of Financial Instruments and Classification Within the Fair Value Hierarchy',
        r'Fair Values of Financial Instruments by Instrument Classification',
        r'Fair Values of Financial Instruments by Instrument Classification',
        r'Financial Instruments Measured at Fair Value on a Recurring Basis',
        r'Financial Instruments Not Measured at Fair Value',
        r'Reconciliation of Level 3 Fair Values',
        r'Prepaid Expenses and Other Tax Assets',
        r'Provisions for Interest and Penalties Related to Taxes',
        r'Other Tax Liabilities',
        r'Executive Board Compensation',
        r'Retirement Pension Plan for Executive Board Members',
        r'Related Party Transactions',
        r'Major Subsidiaries',
        r'Five-Year Summary'
    ]

### function to Removes headers, footers, page numbers, and other noise from the text.
    

In [ ]:
def clean_page_text(text):

    lines = text.split('\n')
    cleaned_lines = []

    for line in lines:
        line = line.strip()

        # Skip empty lines
        if not line:
            continue

        # Remove common header/footer patterns
        if re.match(r'^\s*page\s*\d+(\s*of\s*\d+)?\s*$', line, re.IGNORECASE):
            continue
        if re.match(r'^.*SAP.*\d{4}.*$', line, re.IGNORECASE): 
            continue
        if re.match(r'^(Table of Contents|Contents)$', line.strip(), re.IGNORECASE):
            continue
        if re.match(r'^\d{1,2}$', line.strip()):
            continue
        if len(line) < 5 and line.isdigit():
            continue

        cleaned_lines.append(line)

    return '\n'.join(cleaned_lines)

### Generated Question & Answers from csv folder 

In [4]:
def generate_qa_from_csv_folder(csv_folder, output_csv, max_per_table=10):
    qa_pairs = []
    for fname in os.listdir(csv_folder):
        if not fname.endswith('.csv'):
            continue
        df = pd.read_csv(os.path.join(csv_folder, fname))
        for i, row in df.iterrows():
            if i >= max_per_table:
                break
            for col in df.columns[1:]:
                question = f"What was the {col} for {row[df.columns[0]]}?"
                answer = f"The {col} for {row[df.columns[0]]} was {row[col]}."
                qa_pairs.append((question, answer))
    with open(output_csv, "w", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Question", "Answer"])
        writer.writerows(qa_pairs)

### Extracts markdown tables and returns as list of DataFrames

In [5]:
def extract_tables_from_md(md_path):

    with open(md_path, 'r', encoding='utf-8') as f:
        text = f.read()
    tables = re.findall(r'((?:\|.*\n)+)', text)
    dfs = []
    for table in tables:
        try:
            df = pd.read_csv(pd.compat.StringIO(table), sep='|', engine='python').dropna(axis=1, how='all')
            if len(df.columns) > 1:
                dfs.append(df)
        except Exception:
            continue
    return dfs

### Extracts Q&A from lines with numbers (e.g., "Revenue: $4.13 billion")

In [6]:
def extract_simple_qa_from_text(md_path, max_pairs=50):

    qa_pairs = []
    with open(md_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or ':' not in line:
                continue
            key, value = line.split(':', 1)
            if re.search(r'\d', value):
                question = f"What is the {key.strip()}?"
                answer = f"The {key.strip()} is {value.strip()}."
                qa_pairs.append((question, answer))
            if len(qa_pairs) >= max_pairs:
                break
    return qa_pairs

### Generate Q&A Pairs from Markdown files

In [ ]:
def generate_qa_from_md(md_path, output_csv, max_pairs=50):
    qa_pairs = []
    all_pairs = []

    dfs = extract_tables_from_md(md_path)
    for df in dfs:
        for i, row in df.iterrows():
            for col in df.columns[1:]:
                question = f"What was the {col.strip()} for {row[df.columns[0]].strip()}?"
                answer = f"The {col.strip()} for {row[df.columns[0]].strip()} was {row[col]}."
                all_pairs.append((question, answer))

    all_pairs += extract_simple_qa_from_text(md_path, max_pairs * 2)
    seen_templates = set()
    diverse_pairs = []
    for q, a in all_pairs:
        template = remove_years_numbers(q)
        if template not in seen_templates:
            seen_templates.add(template)
            diverse_pairs.append((q, a))
        if len(diverse_pairs) >= max_pairs:
            break

    if len(diverse_pairs) < max_pairs:
        for q, a in all_pairs:
            if (q, a) not in diverse_pairs:
                diverse_pairs.append((q, a))
            if len(diverse_pairs) >= max_pairs:
                break

    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    with open(output_csv, "w", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Question", "Answer"])
        writer.writerows(diverse_pairs[:max_pairs])

### Extract Segaments from pdf

In [8]:
def extract_segment_tables_from_pdf(pdf_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, 1):
            text = page.extract_text() or ""
            for header in section_table_headers:
                # Search for the header in the page text (case-insensitive)
                if re.search(re.escape(header), text, re.IGNORECASE):
                    tables = page.extract_tables()
                    if tables:
                        # Save the first table found after the header
                        safe_header = re.sub(r'[^\w\d_]+', '_', header)[:50]
                        csv_path = os.path.join(output_dir, f"{safe_header}_page{page_num}.csv")
                        with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
                            writer = csv.writer(csvfile)
                            for row in tables[0]:
                                writer.writerow(row)
                        print(f"Extracted table for '{header}' on page {page_num} to {csv_path}")
    print("Done extracting segment tables.")

### Extract images from pdf using ocr and save to image folder

In [9]:
def pdf_to_markdown_with_easyocr(pdf_path, output_md_path, image_folder_name="images", do_ocr=True):

    # Optional OCR
    reader = None
    if do_ocr:
        import easyocr
        reader = easyocr.Reader(['en'])  # You can add other languages

    # Fallback cleaner if not defined
    def clean_page_text_default(text):
        return " ".join(text.split())  # simple whitespace cleaner

    clean_func = globals().get("clean_page_text", clean_page_text_default)

    # Output dirs
    pdf_dir = os.path.dirname(os.path.abspath(pdf_path))
    image_dir = os.path.join(pdf_dir, image_folder_name)
    if do_ocr:
        os.makedirs(image_dir, exist_ok=True)

    md_lines = []

    # Step 1: Extract text from PDF directly
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""
            cleaned_text = clean_func(text)
            if cleaned_text.strip():
                md_lines.append(cleaned_text.strip() + "\n")

    # Step 2: OCR images from PDF (optional)
    if do_ocr:
        doc = fitz.open(pdf_path)
        for page_num in range(len(doc)):
            page = doc[page_num]
            for img_index, img in enumerate(page.get_images(full=True)):
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]
                image_filename = f"page{page_num + 1}_img{img_index + 1}.{image_ext}"
                image_path = os.path.join(image_dir, image_filename)

                with open(image_path, "wb") as f:
                    f.write(image_bytes)

                pil_image = Image.open(BytesIO(image_bytes)).convert("RGB")
                img_np = np.array(pil_image)  # ✅ Convert to NumPy array

                ocr_result = reader.readtext(img_np)
                ocr_text = " ".join([text for _, text, _ in ocr_result])
                cleaned_ocr = clean_func(ocr_text)

                if cleaned_ocr:
                    md_lines.append(f"> {cleaned_ocr}\n")

    # Step 3: Save markdown
    with open(output_md_path, "w", encoding="utf-8") as f:
        for line in md_lines:
            f.write(line + "\n")

    return output_md_path


In [10]:
def extract_segment_tables_from_pdf1(pdf_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, 1):
            text = page.extract_text() or ""
            for header in section_table_headers:
                # Find header position in text
                header_match = re.search(re.escape(header), text, re.IGNORECASE)
                if not header_match:
                    continue
                tables = page.extract_tables()
                if not tables:
                    continue
                # Try to find the table closest to the header
                # (pdfplumber does not provide table positions, so we take the first table)
                table = tables[0]
                if not table or not any(row for row in table):
                    continue
                # Clean up table: remove empty rows
                table = [row for row in table if any(cell and str(cell).strip() for cell in row)]
                if not table:
                    continue
                safe_header = re.sub(r'[^\w\d_]+', '_', header)[:50]
                csv_path = os.path.join(output_dir, f"{safe_header}_page{page_num}.csv")
                with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
                    writer = csv.writer(csvfile)
                    for row in table:
                        writer.writerow([cell if cell is not None else "" for cell in row])
                print(f"Extracted table for '{header}' on page {page_num} to {csv_path}")
    print("Done extracting segment tables.")
def remove_years_numbers(text):
    # Remove years (4-digit numbers) and all standalone numbers
    return re.sub(r'\b\d{4}\b|\b\d+\b', '', text).strip().lower()

### Util functions

In [11]:
def remove_years_numbers(text):
    # Remove years (4-digit numbers) and all standalone numbers
    return re.sub(r'\b\d{4}\b|\b\d+\b', '', text).strip().lower()


def segment_report(md_path, summary_path):
    # Define section headers to segment the report


    # Read the cleaned Markdown file
    with open(md_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # Build regex pattern for section headers
    pattern = '|'.join([f'({h})' for h in section_table_headers])
    sections = re.split(pattern, text)

    # Organize into a dictionary
    report_sections = {}
    current_section = 'Preamble'
    buffer = []
    for part in sections:
        if part is None:
            continue
        part = part.strip()
        if part in section_table_headers:
            if buffer:
                report_sections[current_section] = '\n'.join(buffer).strip()
                buffer = []
            current_section = part
        else:
            buffer.append(part)
    if buffer:
        report_sections[current_section] = '\n'.join(buffer).strip()

    # Write to summary file
    with open(summary_path, 'w', encoding='utf-8') as f:
        for section, content in report_sections.items():
            f.write(f"# {section}\n\n{content}\n\n")

### Processing pdf conver to markdown file and extract images (ocr) generate Q&A Pairs

In [34]:
print("Processing file ../data/NYSE_SAP_2023.pdf")
pdf_path = "../data/NYSE_SAP_2023.pdf" 
md_path = "../data/md/NYSE_SAP_2023.md"
summary_path = "../data/md/NYSE_SAP_2023_summary.md"
csv_output = "../data/csv/NYSE_SAP_2023"

os.makedirs("../data/md", exist_ok=True)
os.makedirs("../data/csv", exist_ok=True)
os.makedirs("../data/Q&A", exist_ok=True)
# Convert PDF to Markdown with OCR
pdf_to_markdown_with_easyocr(pdf_path, md_path,image_folder_name="images", do_ocr=False)
# extract_tables_from_pdf("../data/NYSE_SAP_2023.pdf", "../data/csv")
segment_report('../data/md/NYSE_SAP_2023.md', '../data/md/segaments_summary_2023.md')
extract_segment_tables_from_pdf(pdf_path, csv_output)
print("Extracted segment tables from PDF.")
print("Processed file ../data/NYSE_SAP_2023.pdf")
print("Processing file ../data/NYSE_SAP_2024.pdf")
pdf_path = "../data/NYSE_SAP_2024.pdf"
md_path = "../data/md/NYSE_SAP_2024.md"
summary_path = "../data/md/NYSE_SAP_2024_summary.md"
csv_output = "../data/csv/NYSE_SAP_2024"

os.makedirs("../data/md", exist_ok=True)
os.makedirs("../data/csv", exist_ok=True)
# Convert PDF to Markdown with OCR
pdf_to_markdown_with_easyocr(pdf_path, md_path,image_folder_name="images", do_ocr=False)


# Example usage:
segment_report('../data/md/NYSE_SAP_2023.md', '../data/md/segaments_summary_2024.md')
extract_segment_tables_from_pdf(pdf_path, csv_output)
print("Processed file ../data/NYSE_SAP_2024.pdf")
years = ["2023", "2024"]
generate_qa_from_md("../data/md/NYSE_SAP_2023.md", "../data/Q&A/NYSE_SAP_2023.csv")
generate_qa_from_md("../data/md/NYSE_SAP_2024.md", "../data/Q&A/NYSE_SAP_2024.csv")
print("Generated Q&A from Markdown files.")

Processing file ../data/NYSE_SAP_2023.pdf
Extracted table for 'Integrated Report' on page 2 to ../data/csv/NYSE_SAP_2023/Integrated_Report_page2.csv
Extracted table for 'Integrated Report' on page 11 to ../data/csv/NYSE_SAP_2023/Integrated_Report_page11.csv
Extracted table for 'Key Facts About SAP Stock/SAP ADRs' on page 12 to ../data/csv/NYSE_SAP_2023/Key_Facts_About_SAP_Stock_SAP_ADRs_page12.csv
Extracted table for 'Integrated Report' on page 12 to ../data/csv/NYSE_SAP_2023/Integrated_Report_page12.csv
Extracted table for 'Integrated Report' on page 13 to ../data/csv/NYSE_SAP_2023/Integrated_Report_page13.csv
Extracted table for 'Integrated Report' on page 16 to ../data/csv/NYSE_SAP_2023/Integrated_Report_page16.csv
Extracted table for 'Integrated Report' on page 21 to ../data/csv/NYSE_SAP_2023/Integrated_Report_page21.csv
Extracted table for 'Executive Board Compensation' on page 21 to ../data/csv/NYSE_SAP_2023/Executive_Board_Compensation_page21.csv
Extracted table for 'Integrated 

### 2. Retrieval-Augmented Generation (RAG) System Implementation
```
# 2.1 Data Processing
#
# Split the cleaned text into chunks suitable for retrieval with at least two chunk sizes (e.g., 100 and 400 tokens).
# Assign unique IDs and metadata to chunks.
# 2.2 Embedding & Indexing
#
# Embed chunks using a small open-source sentence embedding model (e.g., all-MiniLM-L6-v2, E5-small-v2).
# Build:
# Dense vector store (e.g., FAISS, ChromaDB).
# Sparse index (BM25 or TF-IDF) for keyword retrieval.

### DATA PROCESSING SCRIPT AND EMBEDDING FOR RAG PIPELINE ###

In [10]:
MD_DIR = '../data/md'
CHUNKS_PATH = '../data/embeddings/chunks.json'
FAISS_INDEX_PATH = '../data/embeddings/faiss_index.idx'
TFIDF_VECTORIZER_PATH = '../data/embeddings/tfidf_vectorizer.joblib'
QNA_DIR = '../data/Q&A'
MEMORY_BANK_PATH = '../data/embeddings/memory_bank.json'
TOP_N = 10

In [ ]:
# %pip install --no-cache-dir numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 7.1 MB/s eta 0:00:0000:0100:01m
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.1.10 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.11.7 which is incompatible.
spacy 3.8.7 requires thinc<8.4.0,>=8.3.4, but you have thinc 8.1.10 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import glob
import uuid
import json
import re
import pandas as pd
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import heapq

/opt/anaconda3/envs/llmenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Preprocessing before embedding

In [3]:
def clean_text(text: str) -> str:
    text = re.sub(r'```.*?```', '', text, flags=re.DOTALL) 
    text = re.sub(r'`.*?`', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def split_text(text: str, chunk_size: int) -> List[str]:
    tokens = text.split()
    return [' '.join(tokens[i:i+chunk_size]) for i in range(0, len(tokens), chunk_size)]

def process_chunks(text: str, chunk_sizes=[100, 400]) -> List[Dict]:
    chunks = []
    for size in chunk_sizes:
        for idx, chunk in enumerate(split_text(text, size)):
            chunks.append({
                'id': str(uuid.uuid4()),
                'chunk': chunk,
                'size': size,
                'meta': {'chunk_idx': idx, 'chunk_size': size}
            })
    return chunks

### Emebed the retrived chunks using embedding model all-MiniLM-L6-v2

In [4]:
def embed_chunks(chunks: List[Dict], model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = model.encode([c['chunk'] for c in chunks], show_progress_bar=True)
    return embeddings



### Rerank the retrived chunks using similarity score with query

In [5]:
def rerank_chunks(query, candidate_chunks, embedder, top_k=5):
    query_emb = embedder.encode([query])[0]
    chunk_embs = embedder.encode([c['chunk'] for c in candidate_chunks])
    sims = [np.dot(query_emb, emb) / (np.linalg.norm(query_emb) * np.linalg.norm(emb) + 1e-8) for emb in chunk_embs]
    ranked = sorted(zip(candidate_chunks, sims), key=lambda x: -x[1])
    return [c for c, _ in ranked[:top_k]]

### Build FAISS DB

In [6]:
def build_faiss_index(embeddings: np.ndarray):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings.astype('float32'))
    return index

### Build TF-IDF Vectorizer

In [7]:
def build_tfidf_index(chunks: List[Dict]):
    texts = [c['chunk'] for c in chunks]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    return vectorizer, tfidf_matrix

## Methods for 2.3 Hybrid Retrieval Pipeline ####

#### preprocessing

In [8]:
def preprocess(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = text.lower().split()
    tokens = [t for t in tokens if t not in ENGLISH_STOP_WORDS]
    return ' '.join(tokens)

In [11]:
def load_chunks():
    with open(CHUNKS_PATH, 'r', encoding='utf-8') as f:
        return json.load(f)

def dense_retrieve(query, embedder, faiss_index, chunk_ids, top_n=TOP_N):
    q_emb = embedder.encode([query])
    D, I = faiss_index.search(np.array(q_emb).astype('float32'), top_n)
    return [(chunk_ids[i], D[0][rank]) for rank, i in enumerate(I[0])]

In [12]:
def sparse_retrieve(query, tfidf_vectorizer, tfidf_matrix, chunk_ids, top_n=TOP_N):
    q_vec = tfidf_vectorizer.transform([query])
    scores = (tfidf_matrix @ q_vec.T).toarray().squeeze()
    top_idx = heapq.nlargest(top_n, range(len(scores)), scores.take)
    return [(chunk_ids[i], scores[i]) for i in top_idx]

### Weighted score fusion: alpha*dense + (1-alpha)*sparse

In [13]:
def combine_results(dense, sparse, top_n=TOP_N, alpha=0.5):
    scores = {}
    for cid, score in dense:
        scores[cid] = scores.get(cid, 0) + alpha * score
    for cid, score in sparse:
        scores[cid] = scores.get(cid, 0) + (1 - alpha) * score
    sorted_chunks = sorted(scores.items(), key=lambda x: -x[1])[:top_n]
    return [cid for cid, _ in sorted_chunks]

### 2.4 Advanced RAG Technique (Select One)
#### 2.4 ADVANCED RAG Technique (Group 100%5==0)


In [14]:
def load_memory_bank():
    if os.path.exists(MEMORY_BANK_PATH):
        with open(MEMORY_BANK_PATH, 'r', encoding='utf-8') as f:
            return json.load(f)
    return []

def save_to_memory_bank(query, llm_output, embedder):
    memory_bank = load_memory_bank()
    query_clean = preprocess(query)
    query_emb = embedder.encode([query_clean])[0].tolist()
    memory_bank.append({
        'query': query_clean,
        'query_emb': query_emb,
        'llm_output': llm_output
    })
    with open(MEMORY_BANK_PATH, 'w', encoding='utf-8') as f:
        json.dump(memory_bank, f, ensure_ascii=False, indent=2)

### Search from memory bank

In [15]:
def memory_bank_match(query, embedder, memory_bank, threshold=0.9):
    query_clean = preprocess(query)
    query_emb = embedder.encode([query_clean])[0]
    for item in memory_bank:
        mem_emb = np.array(item['query_emb'])
        sim = np.dot(query_emb, mem_emb) / (np.linalg.norm(query_emb) * np.linalg.norm(mem_emb) + 1e-8)
        if sim > threshold:
            return item['llm_output']
    return None

### Update the momory bank

In [16]:
def update_memory_bank(query, llm_output, embedder, memory_bank_path):
    # Load existing memory bank
    if os.path.exists(memory_bank_path):
        with open(memory_bank_path, 'r', encoding='utf-8') as f:
            memory_bank = json.load(f)
    else:
        memory_bank = []
    # Preprocess and embed
    query_clean = preprocess(query)
    query_emb = embedder.encode([query_clean])[0].tolist()
    # Append new entry
    memory_bank.append({
        'query': query_clean,
        'query_emb': query_emb,
        'llm_output': llm_output
    })
    # Save updated memory bank
    with open(memory_bank_path, 'w', encoding='utf-8') as f:
        json.dump(memory_bank, f, ensure_ascii=False, indent=2)

### Preprocessing and Embedding 
### 2.2 Embedding & Indexing

In [18]:
all_text = ""
for md_file in glob.glob(os.path.join(MD_DIR, '*.md')):
    with open(md_file, 'r', encoding='utf-8') as f:
        all_text += clean_text(f.read()) + "\n"

chunks = process_chunks(all_text)
with open(CHUNKS_PATH, 'w', encoding='utf-8') as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

embeddings = embed_chunks(chunks)
faiss_index = build_faiss_index(np.array(embeddings))
faiss.write_index(faiss_index, FAISS_INDEX_PATH)

tfidf_vectorizer, tfidf_matrix = build_tfidf_index(chunks)
joblib.dump(tfidf_vectorizer, TFIDF_VECTORIZER_PATH)

print(f"Processed {len(chunks)} chunks from markdown files.")

Batches: 100%|██████████| 154/154 [00:20<00:00,  7.50it/s]


Processed 4919 chunks from markdown files.
